<a href="https://colab.research.google.com/github/ji-hun-choi/SOTA-Study/blob/main/mobilenetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/d-li14/mobilenetv2.pytorch.git

fatal: destination path 'mobilenetv2.pytorch' already exists and is not an empty directory.


In [ ]:
!nvidia-smi

Tue Oct 11 01:30:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
ls

mobilenetv2.pytorch/  sample_data/


In [ ]:
cd mobilenetv2.pytorch/

/content/mobilenetv2.pytorch


In [9]:
ls

imagenet.py  LICENSE  models/  pretrained/  README.md  utils/


In [ ]:
import torch
from models.imagenet import mobilenetv2

## model load

In [36]:
model = mobilenetv2()
# print(model)
model.load_state_dict(torch.load('./pretrained/mobilenetv2-c5e733a8.pth'))

<All keys matched successfully>

## dataset load

In [21]:
import gdown
## google drive에서 압축된 dataset download
url = 'https://drive.google.com/uc?id=1DtxpOUpARxcnU4qHINz9OgiOdXiji1RZ'
fname = 'hym_data.zip'
gdown.download(url, fname, quiet=False)
!unzip hym_data.zip # test를 위한 data.zip 파일 가져오기.

Downloading...
From: https://drive.google.com/uc?id=1DtxpOUpARxcnU4qHINz9OgiOdXiji1RZ
To: /content/mobilenetv2.pytorch/hym_data.zip
100%|██████████| 95.0M/95.0M [00:00<00:00, 164MB/s]


Archive:  hym_data.zip
  inflating: hym_data/val/ants/183260961_64ab754c97.jpg  
  inflating: hym_data/val/ants/desert_ant.jpg  
  inflating: hym_data/val/ants/Ant-1818.jpg  
  inflating: hym_data/val/ants/436944325_d4925a38c7.jpg  
  inflating: hym_data/val/ants/768870506_8f115d3d37.jpg  
  inflating: hym_data/val/ants/412436937_4c2378efc2.jpg  
  inflating: hym_data/val/ants/11381045_b352a47d8c.jpg  
  inflating: hym_data/val/ants/649407494_9b6bc4949f.jpg  
  inflating: hym_data/val/ants/119785936_dd428e40c3.jpg  
  inflating: hym_data/val/ants/159515240_d5981e20d1.jpg  
  inflating: hym_data/val/ants/1119630822_cd325ea21a.jpg  
  inflating: hym_data/val/ants/172772109_d0a8e15fb0.jpg  
  inflating: hym_data/val/ants/540543309_ddbb193ee5.jpg  
  inflating: hym_data/val/ants/F.pergan.28(f).jpg  
  inflating: hym_data/val/ants/2255445811_dabcdf7258.jpg  
  inflating: hym_data/val/ants/459442412_412fecf3fe.jpg  
  inflating: hym_data/val/ants/263615709_cfb28f6b8e.jpg  
  inflating: hym_d

In [23]:
import os
import time
import copy
import numpy as np
import matplotlib.pyplot as plt
import torch

import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, models, transforms

In [43]:
ddir = '/content/mobilenetv2.pytorch/hym_data'
batch_size = 4
num_workers = 2
data_transformers = {
    'train': transforms.Compose(
        [
         transforms.RandomResizedCrop(224), 
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411], [0.231, 0.221, 0.230])
        ]
    ),
    'val': transforms.Compose(
        [
         transforms.Resize(256),
         transforms.CenterCrop(224),
         transforms.ToTensor(),
         transforms.Normalize([0.490, 0.449, 0.411],[0.231, 0.221, 0.230])
        ]
    )
}

img_data = {
    k:datasets.ImageFolder(os.path.join(ddir, k), data_transformers[k])
    for k in ['train', 'val']
}
dloaders = {
    k: torch.utils.data.DataLoader(
        img_data[k], batch_size=batch_size, shuffle=True, num_workers=num_workers
    )
    for k in ['train', 'val']
}
dset_sizes = {x: len(img_data[x]) for x in ['train', 'val']}
classes = img_data['train'].classes
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [44]:
def train(model, loss_func, optimizer, epochs=10):
    start = time.time()

    accuracy = 0.0

    for e in range(epochs):
        print(f'Epoch number {e}/{epochs - 1}')
        print("=" * 20)

        for dset in ['train', 'val']:
            if dset == 'train':
                model.train()
            else:
                model.eval()
            loss = 0.0
            successes = 0

            for imgs, tgts in dloaders[dset]:
                imgs = imgs.to(device)
                tgts = tgts.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(dset == 'train'):
                    ops = model(imgs)
                    _, preds = torch.max(ops, 1)
                    loss_curr = loss_func(ops, tgts)
                    
                    if dset == 'train':
                        loss_curr.backward()
                        optimizer.step()
                
                loss += loss_curr.item() * imgs.size(0)
                successes += torch.sum(preds == tgts.data)

            loss_epoch = loss / dset_sizes[dset]
            accuracy_epoch = successes.double() / dset_sizes[dset]

            print(f'{dset} loss in this epoch: {loss_epoch}, accuracy in this epoch: {accuracy_epoch}')
            if dset == 'val' and accuracy_epoch > accuracy:
                accuracy = accuracy_epoch
    time_delta = time.time() - start
    print(f'Training finished in {time_delta // 60}mins {time_delta % 60} secs')
    print(f'Best validation set acuuracy: {accuracy}')
    return model

In [69]:
def test(model):

    correct_pred = {classname: 0 for classname in classes}
    total_pred = {classname: 0 for classname in classes}


    with torch.no_grad():
        for images, labels in dloaders['val']:


            images = images.to(device)
            labels = labels.to(device)
        
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            for label, prediction in zip(labels, predictions):
                if label == prediction:
                    correct_pred[classes[label]] += 1
                total_pred[classes[label]] += 1


    for classname, correct_count in correct_pred.items():
        accuracy = 100 * float(correct_count) / total_pred[classname]
        print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

In [51]:
test(model)

Accuracy for class: ants  is 71.4 %
Accuracy for class: bees  is 43.4 %


In [52]:
for name, param in model.named_parameters():
    print('layer name:', name)

layer name: features.0.0.weight
layer name: features.0.1.weight
layer name: features.0.1.bias
layer name: features.1.conv.0.weight
layer name: features.1.conv.1.weight
layer name: features.1.conv.1.bias
layer name: features.1.conv.3.weight
layer name: features.1.conv.4.weight
layer name: features.1.conv.4.bias
layer name: features.2.conv.0.weight
layer name: features.2.conv.1.weight
layer name: features.2.conv.1.bias
layer name: features.2.conv.3.weight
layer name: features.2.conv.4.weight
layer name: features.2.conv.4.bias
layer name: features.2.conv.6.weight
layer name: features.2.conv.7.weight
layer name: features.2.conv.7.bias
layer name: features.3.conv.0.weight
layer name: features.3.conv.1.weight
layer name: features.3.conv.1.bias
layer name: features.3.conv.3.weight
layer name: features.3.conv.4.weight
layer name: features.3.conv.4.bias
layer name: features.3.conv.6.weight
layer name: features.3.conv.7.weight
layer name: features.3.conv.7.bias
layer name: features.4.conv.0.weig

In [53]:
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 2)

In [54]:
test(model)

Accuracy for class: ants  is 21.4 %
Accuracy for class: bees  is 91.6 %


In [55]:
"""
Creates a MobileNetV2 Model as defined in:
Mark Sandler, Andrew Howard, Menglong Zhu, Andrey Zhmoginov, Liang-Chieh Chen. (2018). 
MobileNetV2: Inverted Residuals and Linear Bottlenecks
arXiv preprint arXiv:1801.04381.
import from https://github.com/tonylins/pytorch-mobilenet-v2
"""

import torch.nn as nn
import math

__all__ = ['mobilenetv2']


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    :param v:
    :param divisor:
    :param min_value:
    :return:
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def conv_3x3_bn(inp, oup, stride):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


def conv_1x1_bn(inp, oup):
    return nn.Sequential(
        nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
        nn.BatchNorm2d(oup),
        nn.ReLU6(inplace=True)
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio):
        super(InvertedResidual, self).__init__()
        assert stride in [1, 2]

        hidden_dim = round(inp * expand_ratio)
        self.identity = stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                nn.Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # dw
                nn.Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                nn.BatchNorm2d(hidden_dim),
                nn.ReLU6(inplace=True),
                # pw-linear
                nn.Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
            )

    def forward(self, x):
        if self.identity:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(nn.Module):
    def __init__(self, num_classes=1000, width_mult=1.):
        super(MobileNetV2, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = [
            # t, c, n, s
            [1,  16, 1, 1],
            [6,  24, 2, 2],
            [6,  32, 3, 2],
            [6,  64, 4, 2],
            [6,  96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        input_channel = _make_divisible(32 * width_mult, 4 if width_mult == 0.1 else 8)
        layers = [conv_3x3_bn(3, input_channel, 2)]
        # building inverted residual blocks
        block = InvertedResidual
        for t, c, n, s in self.cfgs:
            output_channel = _make_divisible(c * width_mult, 4 if width_mult == 0.1 else 8)
            for i in range(n):
                layers.append(block(input_channel, output_channel, s if i == 0 else 1, t))
                input_channel = output_channel
        self.features = nn.Sequential(*layers)
        # building last several layers
        output_channel = _make_divisible(1280 * width_mult, 4 if width_mult == 0.1 else 8) if width_mult > 1.0 else 1280
        self.conv = conv_1x1_bn(input_channel, output_channel)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(output_channel, num_classes)

        self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

def mobilenetv2(**kwargs):
    """
    Constructs a MobileNet V2 model
    """
    return MobileNetV2(**kwargs)


In [56]:
net = MobileNetV2()

In [57]:
test(net)

Accuracy for class: ants  is 0.0 %
Accuracy for class: bees  is 0.0 %


In [58]:
net.load_state_dict(torch.load('pretrained/mobilenetv2-c5e733a8.pth'))

<All keys matched successfully>

In [59]:
test(net)

Accuracy for class: ants  is 0.0 %
Accuracy for class: bees  is 0.0 %


In [64]:
for name, param in net.named_parameters():
    print('layer name:', name)

layer name: features.0.0.weight
layer name: features.0.1.weight
layer name: features.0.1.bias
layer name: features.1.conv.0.weight
layer name: features.1.conv.1.weight
layer name: features.1.conv.1.bias
layer name: features.1.conv.3.weight
layer name: features.1.conv.4.weight
layer name: features.1.conv.4.bias
layer name: features.2.conv.0.weight
layer name: features.2.conv.1.weight
layer name: features.2.conv.1.bias
layer name: features.2.conv.3.weight
layer name: features.2.conv.4.weight
layer name: features.2.conv.4.bias
layer name: features.2.conv.6.weight
layer name: features.2.conv.7.weight
layer name: features.2.conv.7.bias
layer name: features.3.conv.0.weight
layer name: features.3.conv.1.weight
layer name: features.3.conv.1.bias
layer name: features.3.conv.3.weight
layer name: features.3.conv.4.weight
layer name: features.3.conv.4.bias
layer name: features.3.conv.6.weight
layer name: features.3.conv.7.weight
layer name: features.3.conv.7.bias
layer name: features.4.conv.0.weig

In [66]:
num_ftrs = net.classifier.in_features
net.classifier = nn.Linear(num_ftrs, 2)

In [68]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)
train(net.cuda(), loss_func, optimizer, epochs=5)

Epoch number 0/4
train loss in this epoch: 0.5341746614116137, accuracy in this epoch: 0.7254098360655739
val loss in this epoch: 0.258012643327413, accuracy in this epoch: 0.954248366013072
Epoch number 1/4
train loss in this epoch: 0.4215929685923897, accuracy in this epoch: 0.8032786885245902
val loss in this epoch: 0.20846138361330127, accuracy in this epoch: 0.934640522875817
Epoch number 2/4
train loss in this epoch: 0.4183538207998041, accuracy in this epoch: 0.7950819672131147
val loss in this epoch: 0.18456276133656502, accuracy in this epoch: 0.934640522875817
Epoch number 3/4
train loss in this epoch: 0.2798727068011878, accuracy in this epoch: 0.8606557377049181
val loss in this epoch: 0.211202210058866, accuracy in this epoch: 0.9150326797385622
Epoch number 4/4
train loss in this epoch: 0.3778269762387041, accuracy in this epoch: 0.8483606557377049
val loss in this epoch: 0.17503949283056108, accuracy in this epoch: 0.954248366013072
Training finished in 0.0mins 26.057316

MobileNetV2(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
        (3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU6(inplace=True)
       

In [70]:
test(net.cuda())

Accuracy for class: ants  is 95.7 %
Accuracy for class: bees  is 95.2 %
